In [ ]:
import os
import pandas as pd

# --- 設定項目 ---
# 処理対象のルートディレクトリ (元のデータが入っている場所)
ROOT_DIR = './' 
# 分割したウィンドウを保存する新しいルートディレクトリ
OUTPUT_ROOT_DIR = 'work_windowed'

# 【変更点1】データトリミングの時間を定義できるようにする
TRIM_START_SECONDS = 0.0  # 計測開始後、トリミングする秒数
TRIM_END_SECONDS = 0.0    # 計測終了前、トリミングする秒数

# タイムウィンドウの設定
WINDOW_SIZE_SECONDS = 1.0  # 1秒間のウィンドウ
OVERLAP_RATIO = 0.0 # 重複なし
MIN_SAMPLES_PER_WINDOW = 5 

# --- デバッグ情報表示 ---
print("--- デバッグ情報 ---")
print(f"カレントディレクトリ: {os.getcwd()}")
print(f"入力ディレクトリ (ROOT_DIR): {os.path.abspath(ROOT_DIR)}")
print(f"出力ディレクトリ (OUTPUT_ROOT_DIR): {os.path.abspath(OUTPUT_ROOT_DIR)}")
if not os.path.isdir(ROOT_DIR):
    print(f"\n警告: 入力ディレクトリ '{ROOT_DIR}' が見つかりません。")
    print("スクリプトを 'work' ディレクトリと同じ階層に置いて実行していますか？")
print("--------------------")

print(f"\n処理を開始します。")

# 出力用のルートディレクトリを作成
os.makedirs(OUTPUT_ROOT_DIR, exist_ok=True)

# os.walkで指定したディレクトリ以下を再帰的に探索
for dirpath, dirnames, filenames in os.walk(ROOT_DIR):
    print(f"\n[探索中] ディレクトリ: {dirpath}")
    
    # .ipynb_checkpointsのような不要なディレクトリは探索対象から除外
    if '.ipynb_checkpoints' in dirnames:
        dirnames.remove('.ipynb_checkpoints')

    # 処理対象のファイルが3つ揃っているか確認
    if 'x.txt' in filenames and 'y.txt' in filenames and 'z.txt' in filenames:
        print(f"  -> 発見: x.txt, y.txt, z.txt を含むディレクトリです。処理を開始します。")
        
        try:
            # --- ステップ1: 従来の前処理（統合とトリミング） ---
            df_x = pd.read_csv(os.path.join(dirpath, 'x.txt'), header=None, names=['timestamp', 'x'])
            df_y = pd.read_csv(os.path.join(dirpath, 'y.txt'), header=None, names=['timestamp', 'y'])
            df_z = pd.read_csv(os.path.join(dirpath, 'z.txt'), header=None, names=['timestamp', 'z'])
            
            merged_df = df_x.copy()
            merged_df['y'] = df_y['y']
            merged_df['z'] = df_z['z']
            
            print(f"    [前処理] 元データ読み込み完了。行数: {len(merged_df)}")
            if len(merged_df) == 0:
                print("    [警告] 元データが空です。スキップします。")
                continue
                
            merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])
            duration_seconds = (merged_df['timestamp'].iloc[-1] - merged_df['timestamp'].iloc[0]).total_seconds()
            print(f"    [前処理] 元データの時間長: {duration_seconds:.2f} 秒")

            # 【変更点1】トリミング時間の合計を計算し、データ長が足りているかチェック
            total_trim_seconds = TRIM_START_SECONDS + TRIM_END_SECONDS
            if duration_seconds < total_trim_seconds:
                print(f"    [警告] データ長がトリミング時間({total_trim_seconds:.2f}秒)未満のため、スキップしました。")
                continue

            # 【変更点1】設定した秒数でトリミング時間を決定
            start_time = merged_df['timestamp'].iloc[0]
            trim_start_time = start_time + pd.Timedelta(seconds=TRIM_START_SECONDS)
            trim_end_time = merged_df['timestamp'].iloc[-1] - pd.Timedelta(seconds=TRIM_END_SECONDS)
            
            trimmed_df = merged_df[(merged_df['timestamp'] >= trim_start_time) & (merged_df['timestamp'] <= trim_end_time)].copy()
            
            print(f"    [前処理] トリミング後のデータ行数: {len(trimmed_df)}")

            if trimmed_df.empty:
                print(f"    [警告] トリミング後にデータがなくなったため、処理をスキップしました。")
                continue

            new_start_time = trimmed_df['timestamp'].iloc[0]
            trimmed_df['time'] = (trimmed_df['timestamp'] - new_start_time).dt.total_seconds()
            final_df = trimmed_df[['time', 'x', 'y', 'z']]

            # 【変更点2】タイムウィンドウ化する前の時系列データをcsvで保存
            pre_window_base_name = os.path.basename(dirpath) # 例: 'aruki1'
            pre_window_output_filename = f"{pre_window_base_name}.csv" # 例: 'aruki1.csv'
            pre_window_output_path = os.path.join(dirpath, pre_window_output_filename)
            
            final_df.to_csv(pre_window_output_path, index=False, float_format='%.6f')
            print(f"    [保存] ウィンドウ化前のデータを '{pre_window_output_path}' に保存しました。")
            
            # --- ステップ2: タイムウィンドウ処理 ---
            relative_path = os.path.relpath(dirpath, ROOT_DIR)
            output_dir_path = os.path.join(OUTPUT_ROOT_DIR, relative_path)
            os.makedirs(output_dir_path, exist_ok=True)
            print(f"    [ウィンドウ処理] 出力先ディレクトリ: {output_dir_path}")

            all_windows_list = []
            
            window_count = 0
            step_size = WINDOW_SIZE_SECONDS * (1 - OVERLAP_RATIO)
            current_pos = 0.0
            total_duration = final_df['time'].iloc[-1]
            
            print(f"    [ウィンドウ処理] 処理対象の時間長: {total_duration:.2f} 秒")
            
            if total_duration < WINDOW_SIZE_SECONDS:
                 print(f"    [警告] 処理対象の時間長({total_duration:.2f}s)がウィンドウサイズ({WINDOW_SIZE_SECONDS}s)未満のため、ウィンドウを作成できません。")

            while current_pos + WINDOW_SIZE_SECONDS <= total_duration:
                window_df = final_df[
                    (final_df['time'] >= current_pos) & 
                    (final_df['time'] < current_pos + WINDOW_SIZE_SECONDS)
                ].copy()

                if len(window_df) >= MIN_SAMPLES_PER_WINDOW:
                    all_windows_list.append(window_df)
                    window_count += 1
                
                current_pos += step_size

            if all_windows_list:
                combined_df = pd.concat(all_windows_list, ignore_index=True)
                combined_df.drop_duplicates(inplace=True)
                combined_df.sort_values(by='time', inplace=True)
                
                base_name = os.path.basename(dirpath)
                output_filename = f"{base_name}_windowed_all.csv"
                output_path = os.path.join(output_dir_path, output_filename)
                
                combined_df.to_csv(output_path, index=False, float_format='%.6f')
                
                print(f"  -> {window_count}個のウィンドウを結合し、'{output_filename}' に保存しました。")
            else:
                print("  -> 作成された有効なウィンドウはありませんでした。")
        
        except Exception as e:
            print(f"エラー: {dirpath} の処理中にエラーが発生しました - {e}")

print("\nすべての処理が完了しました。")

In [ ]:
import os
import pandas as pd

# 処理対象のルートディレクトリ
root_dir = './'

print(f"処理を開始します。ルートディレクトリ: {root_dir}")

# os.walkで指定したディレクトリ以下を再帰的に探索
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # .ipynb_checkpointsのような不要なディレクトリは探索対象から除外
    if '.ipynb_checkpoints' in dirnames:
        dirnames.remove('.ipynb_checkpoints')

    # 処理対象のファイルが3つ揃っているディレクトリかを判定
    if 'x.txt' in filenames and 'y.txt' in filenames and 'z.txt' in filenames:
        
        print(f"処理中のディレクトリ: {dirpath}")
        
        try:
            # --- 1. 各txtファイルを読み込み、DataFrameを作成 ---
            df_x = pd.read_csv(os.path.join(dirpath, 'x.txt'), header=None, names=['timestamp', 'x'])
            df_y = pd.read_csv(os.path.join(dirpath, 'y.txt'), header=None, names=['timestamp', 'y'])
            df_z = pd.read_csv(os.path.join(dirpath, 'z.txt'), header=None, names=['timestamp', 'z'])
            
            # --- 2. データを統合し、タイムスタンプをdatetimeオブジェクトに変換 ---
            # xのDataFrameをベースに、yとzの値を列として追加
            merged_df = df_x.copy()
            merged_df['y'] = df_y['y']
            merged_df['z'] = df_z['z']
            
            # タイムスタンプの文字列をdatetimeオブジェクトに変換して時間計算を可能にする
            merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])

            # --- 3. データの最初と最後の1秒をトリミング（削除） ---
            # データ全体の長さが2秒未満の場合は処理をスキップ
            if (merged_df['timestamp'].iloc[-1] - merged_df['timestamp'].iloc[0]).total_seconds() < 2:
                print(f"  警告: データ長が2秒未満のため、処理をスキップしました。")
                continue

            # 削除する期間を定義
            start_time = merged_df['timestamp'].iloc[0]
            end_time = merged_df['timestamp'].iloc[-1]
            trim_start_time = start_time + pd.Timedelta(seconds=1)
            trim_end_time = end_time - pd.Timedelta(seconds=1)
            
            # 指定期間内のデータを抽出（.copy()で警告を回避）
            trimmed_df = merged_df[(merged_df['timestamp'] > trim_start_time) & (merged_df['timestamp'] < trim_end_time)].copy()

            # トリミング後にデータが空になった場合はスキップ
            if trimmed_df.empty:
                print(f"  警告: トリミング後にデータがなくなったため、ファイルを作成しませんでした。")
                continue

            # --- 4. 経過時間カラムを作成 ---
            # トリミング後の先頭行の時刻を0秒とする
            new_start_time = trimmed_df['timestamp'].iloc[0]
            trimmed_df['time'] = (trimmed_df['timestamp'] - new_start_time).dt.total_seconds()
            
            # --- 5. 最終的なDataFrameを整形 ---
            # 'timestamp'カラムを削除し、カラムの順序を'time, x, y, z'に指定
            final_df = trimmed_df[['time', 'x', 'y', 'z']]

            # --- 6. 統合・加工したデータをCSVとして保存 ---
            dir_name = os.path.basename(dirpath)
            output_filename = f"{dir_name}_processed.csv" # ファイル名が重複しないように変更
            output_path = os.path.join(dirpath, output_filename)
            
            # index=FalseでDataFrameのインデックスがファイルに書き込まれるのを防ぐ
            # float_formatで小数点以下の桁数を指定
            final_df.to_csv(output_path, index=False, float_format='%.6f')
            
            print(f"  -> 加工済みCSVを作成しました: {output_path}")

        except Exception as e:
            print(f"エラー: {dirpath} の処理中にエラーが発生しました - {e}")

print("\nすべての処理が完了しました。")